## Dependencies

In [45]:
import os
import shutil
import cv2
import glob
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa

## Data Preprocess & Split

In [46]:
keep_stanford40 = ["applauding", "climbing", "drinking", "jumping", "pouring_liquid", "riding_a_bike", "riding_a_horse", 
        "running", "shooting_an_arrow", "smoking", "throwing_frisby", "waving_hands"]

with open('Stanford40/ImageSplits/train.txt', 'r') as f:
    # We won't use these splits but split them ourselves
    train_files = [file_name for file_name in list(map(str.strip, f.readlines())) if '_'.join(file_name.split('_')[:-1]) in keep_stanford40]
    train_labels = ['_'.join(name.split('_')[:-1]) for name in train_files]

    
with open('Stanford40/ImageSplits/test.txt', 'r') as f:
    # We won't use these splits but split them ourselves
    test_files = [file_name for file_name in list(map(str.strip, f.readlines())) if '_'.join(file_name.split('_')[:-1]) in keep_stanford40]
    test_labels = ['_'.join(name.split('_')[:-1]) for name in test_files]

# Combine the splits and split for keeping more images in the training set than the test set.
all_files = train_files + test_files
all_labels = train_labels + test_labels
train_files, test_files = train_test_split(all_files, test_size=0.1, random_state=0, stratify=all_labels)
train_labels = ['_'.join(name.split('_')[:-1]) for name in train_files]
test_labels = ['_'.join(name.split('_')[:-1]) for name in test_files]
action_categories = sorted(list(set(train_labels)))

In [47]:
train_files, val_files = train_test_split(train_files, test_size=0.1, random_state=0, stratify=train_labels, shuffle = True)
train_labels = ['_'.join(name.split('_')[:-1]) for name in train_files]
val_labels = ['_'.join(name.split('_')[:-1]) for name in val_files]

In [48]:
str_to_int = {"applauding": 0, "climbing": 1, "drinking": 2, "jumping":3 , "pouring_liquid":4, "riding_a_bike":5, "riding_a_horse":6, 
              "running":7, "shooting_an_arrow":8, "smoking":9, "throwing_frisby":10, "waving_hands":11}

y_train = np.array([str_to_int[label] for label in train_labels])
y_val = np.array([str_to_int[label] for label in val_labels])
y_test = np.array([str_to_int[label] for label in test_labels])

In [49]:
x_train = []
for i, file in enumerate(train_files):
    img = cv2.imread(os.path.join('Stanford40/JPEGImages/', file))
    img = cv2.resize(img, (112, 112))
    x_train.append(img)
x_train = np.asarray(x_train)/255.

In [50]:
x_val = []
for i, file in enumerate(val_files):
    img = cv2.imread(os.path.join('Stanford40/JPEGImages/', file))
    img = cv2.resize(img, (112, 112))
    x_val.append(img)
x_val = np.asarray(x_val)/255.

In [51]:
x_test = []
for i, file in enumerate(test_files):
    img = cv2.imread(os.path.join('Stanford40/JPEGImages/', file))
    img = cv2.resize(img, (112, 112))
    x_test.append(img)
x_test = np.asarray(x_test)/255.

## Model

In [52]:
# Baseline from Assignment 4
model_no_clr = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(112, 112, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=12, activation='softmax')
])

model_no_clr.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_no_clr.load_weights('weights/stanford.h5')

In [53]:
# print the current learning rate
print("Current learning rate:", model_no_clr.optimizer.lr.numpy())

Current learning rate: 0.001


In [54]:
no_clr_history = model_no_clr.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
19/77 [======>.......................] - ETA: 16s - loss: 0.1756 - accuracy: 0.9441

KeyboardInterrupt: 

In [ ]:
# Function for plotting accuracy over epochs
def plot_acc(history):
    plt.plot(history.history['accuracy'])
    # plt.plot(history.history['val_accuracy'])
    plt.title('Training accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    # plt.legend(['Training', 'Validation'], loc='upper left')
    plt.show()
    
# Function for plotting loss over epochs
def plot_loss(history):
    plt.plot(history.history['loss'])
    # plt.plot(history.history['val_loss'])
    plt.title('Training loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    # plt.legend(['Training', 'Validation'], loc='upper left')
    plt.show()

In [ ]:
plot_acc(no_clr_history)
plot_loss(no_clr_history)

In [ ]:
result = model_no_clr.evaluate(x_test, y_test, batch_size=32)

## Cyclic learning rate

In [ ]:
#initialize hyperparameters for cyclic lr schedule
BATCH_SIZE = 32
EPOCHS = 10
steps_per_epoch = len(x_train) // BATCH_SIZE
INIT_LR = 1e-5
MAX_LR = 1e-3

#initialize the cyclical learning rate scheduler
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * steps_per_epoch
)



In [ ]:
#clr model
model_clr = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(112, 112, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=12, activation='softmax')
])

#optimizer with cyclic learning rate
optimizer = tf.keras.optimizers.SGD(clr)
model_clr.compile(optimizer= optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#print learning rate
print("Current learning rate:", model_clr.optimizer.lr.numpy())

In [ ]:
#plot cyclic lr in steps of 50
step = np.arange(0, EPOCHS * steps_per_epoch)
lr = clr(step)
plt.plot(step, lr)
plt.xlabel("Steps")
plt.ylabel("Learning Rate")
plt.show()

#plot cyclic lr in steps of 100
step = np.arange(0, 100 * steps_per_epoch)
lr = clr(step)
plt.plot(step, lr)
plt.xlabel("Steps")
plt.ylabel("Learning Rate")
plt.show()

In [ ]:
#re-train the model with new learning rate
clr_history = model_clr.fit(x_train, y_train, epochs=10, batch_size=32)

In [ ]:
# Function for plotting accuracy over epochs
def clr_plot_acc(history, history2):
    plt.plot(history.history['accuracy'])
    plt.plot(history2.history['accuracy'])
    plt.title('Training accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['No clr', 'With clr'], loc='upper left')
    plt.show()
    
# Function for plotting loss over epochs
def clr_plot_loss(history, history2):
    plt.plot(history.history['loss'])
    plt.plot(history2.history['loss'])
    plt.title('Training loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['No clr', 'With clr'], loc='upper left')
    plt.show()

In [ ]:
clr_plot_acc(no_clr_history, clr_history)
clr_plot_loss(no_clr_history, clr_history)

In [ ]:
result = model_no_clr.evaluate(x_test, y_test, batch_size=32)